In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
q=0.1
fname = "DATA/dataRBM_q" + str(q) + ".csv"
v = np.loadtxt(fname, delimiter=",", dtype = int)
N = len(v)
L = len(v[0])
print(f"each of N={N} data samples has L={L} digits")
for n in range(10):
    print(v[n])
print("...")

SPINS = True
if SPINS:
    vmin = -1
    GAP = 2
    v = 2*v - 1
else:
    vmin = 0
    GAP = 1
    
#store initial values
v0 = v

for n in range(10):
    print(v[n])
print("...")

each of N=10000 data samples has L=8 digits
[0 1 0 0 1 0 1 1]
[1 1 0 1 1 0 0 1]
[0 1 1 1 0 0 1 1]
[0 0 1 1 1 1 0 0]
[0 0 1 1 0 0 1 0]
[1 1 0 0 1 0 0 0]
[0 1 1 0 1 1 0 0]
[1 1 0 0 0 0 1 1]
[1 0 0 1 1 1 0 1]
[1 0 0 0 0 0 1 0]
...
[-1  1 -1 -1  1 -1  1  1]
[ 1  1 -1  1  1 -1 -1  1]
[-1  1  1  1 -1 -1  1  1]
[-1 -1  1  1  1  1 -1 -1]
[-1 -1  1  1 -1 -1  1 -1]
[ 1  1 -1 -1  1 -1 -1 -1]
[-1  1  1 -1  1  1 -1 -1]
[ 1  1 -1 -1 -1 -1  1  1]
[ 1 -1 -1  1  1  1 -1  1]
[ 1 -1 -1 -1 -1 -1  1 -1]
...


In [3]:
def create_coord(np, x0):
    x = [x0] * np
    y = list(range(np))
    for i in range(np):
        y[i] = y[i]/(np-1.) - 0.5
    return (x, y)

def mycolor(val):
    if val > 0:
        return 'red'
    elif val < 0:
        return 'blue'
    return 'black'

def plotgraph(rbm_res):
    A = 2./rbm_res.w.max()
    (x1, y1) = create_coord(rbm_res.L, 0)
    (x2, y2) = create_coord(rbm_res.M, 1)
    for i in range(rbm_res.L):
        for j in range(rbm_res.M):
            ex, ey, col = (x1[i], x2[j]), (y1[i], y2[j]), mycolor(rbm_res.w[i][j])
            plt.plot(ex, ey, col, zorder = 1, lw=A*np.abs(rbm_res.w[i][j]))
    A = 300./(rbm_res.a.max() + rbm_res.b.max())
    for i in range(rbm_res.L):
        plt.scatter(x1[i], y1[i], s = A*np.abs(rbm_res.a[i]), zorder = 2, c = mycolor(rbm_res.a[i]))
    for j in range(rbm_res.M):
        plt.scatter(x2[j], y2[j], s = A*np.abs(rbm_res.b[j]), zorder = 2, c = mycolor(rbm_res.b[j]))
    plt.title(f">0 red, <0 blue, epoch = {rbm_res.nepochs}")
    plt.show()

In [4]:
import numpy as np

class RBM(object):
    """Random Boltzman Machine

    Parameters
    -----------
    nepochs
    mini
    l_rate
    M

    Attributes
    ----------
    w, a, b
    """

    def __init__(self, l_rate = 1, nepochs = 50, mini = 500, M = 3, beta = 1, random_state = None):
        self.l_rate = l_rate
        self.nepochs = nepochs
        self.mini = mini
        self.M = M
        self.beta = beta
        if random_state:
            np.random.seed(random_state)
    
    def fit(self, v):
        """Train the Network"""
        self.L = v.shape[1]
        N = v.shape[0]
        self._initialize_weights(self.L, self.M)
        GAP = v[0].max() - v[0].min()
        m = 0
        for epoch in range(1, 1+self.nepochs):
            for n in range(N):
                if m==0:
                    #initialize
                    v_data, v_model = np.zeros(self.L), np.zeros(self.L)
                    h_data, h_model = np.zeros(self.M), np.zeros(self.M)
                    vh_data, vh_model = np.zeros((self.L,self.M)), np.zeros((self.L, self.M))

                #positive CD phase
                h = self.activate(v[n], self.w, self.b, GAP)
                #negative CD phase
                vf = self.activate(h, self.w.T, self.a, GAP)
                # positive CD pahse nr 2
                hf = self.activate(vf, self.w, self.b, GAP)

                v_data += v[n]
                v_model += vf
                h_data += h
                h_model += hf
                vh_data += np.outer(v[n].T, h)
                vh_model += np.outer(vf.T, hf)

                m+=1

                if m == self.mini:
                    C = self.l_rate/self.mini
                    self._update_w(vh_data, vh_model, C)
                    self._update_a(v_data, v_model, C)
                    self._update_b(h_data, h_model, C)
                    m = 0
            #randomize order
            np.random.shuffle(v)
            self.l_rate = self.l_rate/(0.05*self.l_rate + 1)
        return self

    def score(self, v, seq):
        N = v.shape[0]
        GAP = v[0].max() - v[0].min()
        v1 = np.full((N, self.L), v.min())
        for n in range(N):
            h = self.activate(v[n],self.w, self.b, GAP)
            v1[n] = self.activate(h, self.w.T, self.a, GAP)
        count = 0
        for n in range(N):
            for i in range(len(seq)):
                if (v1[n] == seq[i]).all():
                    count += 1
                    break
        return count/N

    def _initialize_weights(self):
        sigma = np.sqrt(4./float(self.L+self.M))
        self.w = sigma * (2*np.random.rand(self.L, self.M) - 1)
        self.a = sigma * (2*np.random.rand(self.L) - 1)
        self.b = np.zeros(self.M)
    
    def _update_w(self, data, model, C):
        dw = C*(data-model)
        self.w += dw

    def _update_a(self, data, model, C):
        da = C*(data-model)
        self.a += da

    def _update_b(self, data, model, C):
        db = C*(data-model)
        self.b += db

    def activate(self, v_in, wei, bias, DE):
        act = np.dot(v_in, wei) + bias
        prob = 1./(1.+np.exp(-self.beta*DE*act))
        n = len(act)
        v_out = np.full(n, v_in.min())
        v_out[np.random.random_sample(n) < prob] = 1
        return v_out

In [5]:
my_rbm = RBM(M = 2, random_state=12345)

In [6]:
res = my_rbm.fit(v)

c:\program files\python37\lib\site-packages\ipykernel_launcher.py:105: RuntimeWarning: overflow encountered in exp


In [11]:
res.w

array([[ -25.91356182, -203.68079032],
       [ -22.95822503, -203.69871896],
       [ -58.18936258,    7.64449594],
       [ -59.85419548,    8.93000442],
       [   0.67602049,   -0.91588785],
       [   0.64981465,   -0.76810544],
       [-230.74730213,  -65.80476462],
       [-190.72420044,  -39.52013288]])

In [22]:
plotgraph(res)

ValueError: need at least one array to concatenate

<Figure size 432x288 with 1 Axes>